In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path

import open3d as o3d
import torch
import tqdm
from diffusers import StableDiffusionPipeline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
base_path = Path('../Dataset/')
metadata = pd.read_csv(base_path/'metadata_modelnet40.csv')
dataset_dir = base_path/'ModelNet40'

In [3]:
def read_off(file):
    if 'OFF' != file.readline().strip():
        raise ('Not a valid OFF header')
    n_verts, n_faces, __ = tuple(
        [int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')]
             for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:]
             for i_face in range(n_faces)]
    return verts, faces


# read in data
sample_label = 'car'
with open(f'{dataset_dir}/{sample_label}/train/{sample_label}_0001.off', 'r') as f:
    verts, faces = read_off(f)

In [9]:
# generate texture image
pipe = StableDiffusionPipeline.from_pretrained('../SavedPretrainedModel/texture_diffusion', torch_dtype=torch.float32)
pipe.enable_attention_slicing()
# pipe = StableDiffusionPipeline.from_pretrained('dream-textures/texture-diffusion', torch_dtype=torch.float32)
# pipe.save_pretrained('../SavedPretrainedModel/texture_diffusion')
# pipe = pipe.to('cuda')

prompt = f'{sample_label} texture'
images = pipe(prompt).images
# images[0].save(image_path)

# select a random texture image
image_idx = np.random.choice(len(images))
image_path = f'../Texture/{sample_label}.png'
images[image_idx].save(image_path)

100%|██████████| 50/50 [13:27<00:00, 16.16s/it]


In [5]:
# display the texture image
from IPython.display import Image
Image(filename=image_path)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\wangk\AppData\Local\Temp/ipykernel_6580/3579975076.py:3 in <module>                     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\wangk\\AppData\\Local\\Temp/ipykernel_6580/3579975076.py'                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'image_path' is not defined

In [6]:
def generate_texture_coords(verts, faces, texture_file):
    # Load the texture image
    texture = cv2.imread(texture_file)

    # Create an empty list to store the texture coordinates for each triangle
    triangles_uv = []

    # Loop over each face in the mesh's faces list
    for face in faces:
        # Retrieve the 3D vertex positions corresponding to the three vertices of the current face
        v1, v2, v3 = verts[face[0]], verts[face[1]], verts[face[2]]

        # Compute the texture coordinates for each vertex using the UV image
        uv1 = [v1[0] / texture.shape[1], (texture.shape[0] - v1[1]) / texture.shape[0]]
        uv2 = [v2[0] / texture.shape[1], (texture.shape[0] - v2[1]) / texture.shape[0]]
        uv3 = [v3[0] / texture.shape[1], (texture.shape[0] - v3[1]) / texture.shape[0]]

        # Append the texture coordinates for the three vertices of the current face to the list
        triangles_uv.extend([uv1, uv2, uv3])

    # Convert the list of texture coordinates to a numpy array of shape (num_faces*3, 2)
    triangles_uv = np.array(triangles_uv)

    return texture, triangles_uv

texture, triangles_uv = generate_texture_coords(verts, faces, image_path)
# norma;ize the texture coordinates to the range [0, 1]
triangles_uv = (triangles_uv - np.min(triangles_uv, axis=0)) / (np.max(triangles_uv, axis=0) - np.min(triangles_uv, axis=0))
triangles_uv

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\wangk\AppData\Local\Temp/ipykernel_6580/3512117083.py:26 in <module>                    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\wangk\\AppData\\Local\\Temp/ipykernel_6580/3512117083.py'                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'image_path' is not defined

In [7]:
m = o3d.geometry.TriangleMesh(o3d.open3d.utility.Vector3dVector(verts),
                              o3d.open3d.utility.Vector3iVector(faces))
m.compute_vertex_normals()
m.triangle_uvs = o3d.open3d.utility.Vector2dVector(triangles_uv)
m.triangle_material_ids = o3d.utility.IntVector([0]*len(faces))
m.textures = [o3d.geometry.Image(texture)]

o3d.visualization.draw_geometries([m])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\wangk\AppData\Local\Temp/ipykernel_6580/3135122364.py:4 in <module>                     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\wangk\\AppData\\Local\\Temp/ipykernel_6580/3135122364.py'                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'triangles_uv' is not defined